In [33]:
from math import ceil
import time, re, ast, sys, urllib, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle
from header_list import user_agent_list
from proxies_list import proxies_list_
from INITIAL_SCRAPPER_FUNCTIONS import *
from PARSER_FUNCTIONS import *
from LIST_DF_FUNCTIONS import *
ua = user_agent_list
proxies = proxies_list_

In [13]:
customer_url = input("Input the Redfin URL of the Home: ")

Input the Redfin URL of the Home: 


In [14]:
# customer_df = each_property(customer_url, header, proxy)

In [34]:
customer_df = pd.DataFrame(columns=['address', 'city', 'state', 'zip', 'type',
                                    'price', 'sqft', 'num_bds', 'num_bths',
                                    'lot_sf', 'yr_blt', 'HOA', 'HOA_fee'])

In [35]:
def search_url_part_two_gen(type_home):

    if (type_home == 'Condo' or type_home == 'Townhouse'):
        url_part_two = ',property-type=condo+townhouse'

    elif type_home == 'Single Family House':
        url_part_two = ',property-type=House'

    return url_part_two

In [36]:
def search_url_part_three_gen(price):
    
    if price <= 49999:
        url = ',max-price=75k'

    elif price <= 74999:
        url = ',max-price=100k'

    elif price <= 99999:
        url = ',min-price=50k,max-price=125k'

    elif price <= 124999:
        url = ',min-price=75k,max-price=150k'

    elif price <= 149999:
        url = ',min-price=100k,max-price=175k'

    elif price <= 174999:
        url = ',min-price=125k,max-price=200k'

    elif price <= 199999:
        url = ',min-price=150k,max-price=225k'

    elif price <= 224999:
        url = ',min-price=175k,max-price=250k'

    elif price <= 249999:
        url = ',min-price=200k,max-price=275k'

    elif price <= 274999:
        url = ',min-price=225k,max-price=300k'

    elif price <= 299999:
        url = ',min-price=250k,max-price=325k'

    elif price <= 324999:
        url = ',min-price=275k,max-price=350k'

    elif price <= 349999:
        url = ',min-price=300k,max-price=375k'

    elif price <= 374999:
        url = ',min-price=325k,max-price=400k'

    elif price <= 399999:
        url = ',min-price=350k,max-price=425k'

    elif price <= 424999:
        url = ',min-price=375k,max-price=450k'

    elif price <= 449999:
        url = ',min-price=400k,max-price=475k'

    elif price <= 474999:
        url = ',min-price=425k,max-price=500k'

    elif price <= 499999:
        url = ',min-price=450k,max-price=550k'

    elif price <= 549999:
        url = ',min-price=500k,max-price=600k'

    elif price <= 599999:
        url = ',min-price=500k,max-price=650k'

    elif price <= 649999:
        url = ',min-price=550k,max-price=700k'

    elif price <= 699999:
        url = ',min-price=600k,max-price=750k'

    elif price <= 749999:
        url = ',min-price=650k,max-price=800k'

    elif price <= 799999:
        url = ',min-price=700k,max-price=850k'

    elif price <= 849999:
        url = ',min-price=750k,max-price=900k'

    elif price <= 899999:
        url = ',min-price=800k,max-price=950k'

    elif price <= 949999:
        url = ',min-price=850k,max-price=1M'

    elif price <= 999999:
        url = ',min-price=900k,max-price=1.25M'

    elif price <= 1249999:
        url = ',min-price=950k,max-price=1.5M'

    elif price <= 1499999:
        url = ',min-price=1M,max-price=1.75M'

    elif price <= 1749999:
        url = ',min-price=1.25M,max-price=2M'

    elif price <= 1999999:
        url = ',min-price=1.5M,max-price=2.25M'

    elif price <= 2249999:
        url = ',min-price=1.75M,max-price=2.5M'

    elif price <= 2499999:
        url = ',min-price=2M,max-price=2.75M'

    elif price <= 2749999:
        url = ',min-price=2.25M,max-price=3M'

    elif price <= 2999999:
        url = ',min-price=2.5M,max-price=3.25M'

    elif price <= 3249999:
        url = ',min-price=2.75M,max-price=3.5M'

    elif price <= 3499999:
        url = ',min-price=3M,max-price=4M'

    elif price <= 3749999:
        url = ',min-price=3.25M,max-price=4.25M'

    elif price <= 3999999:
        url = ',min-price=3.5M,max-price=4.5M'

    elif price <= 4249999:
        url = ',min-price=3.75M,max-price=5M'

    elif price <= 4499999:
        url = ',min-price=3.75M,max-price=6M'

    elif price <= 4749999:
        url = ',min-price=4M,max-price=6M'

    elif price <= 4999999:
        url = ',min-price=4.25M,max-price=6M'

    elif price <= 5999999:
        url = ',min-price=4.5M,max-price=7M'

    elif price <= 6999999:
        url = ',min-price=5M,max-price=9M'

    elif price <= 7999999:
        url = ',min-price=5M'

    elif price <= 8999999:
        url = ',min-price=6M'

    else:
        url = ',min-price=7M'

    return url

In [28]:
# def search_url_part_four_gen(price):

#     pmik = int(price//100000)
#     pmvk = int(price % 100000)/1000

#     pmim = int(price//1000000)
#     pmvm = int(price % 1000000)/10000

#     if price <= 500000:
# #         if pmvk <= 24.99:
# #             min_price = str(pmik)+'50k'
#         if pmvk <= 49.99:
#             max_price = str(pmik)+'75k'
#         elif pmvk <= 74.99:
#             max_price = str(pmik+1)+'00k'
#         else:
#             max_price = str(pmik+1)+'25k'
#         url = ',max-price='+max_price

#     elif price <= 900000:

#         if pmvk <= 29.99:
#             max_price = str(pmik)+'50k'
#         elif pmvk <= 59.99:
#             max_price = str(pmik+1)+'00k'
#         else:
#             max_price = str(pmik+1)+'50k'
#         url = ',max-price='+max_price

#     elif price <= 1000000:

#         if pmvk <= 29.99:
#             max_price = '1M'
#         else:
#             max_price = '1.25M'
#         url = ',max-price='+max_price

#     elif price <= 2000000:
#         if pmv <= 24.99:
#             max_price = str(pmim)+'.5M'
#         elif pmv <= 49.99:
#             max_price = str(pmim)+'.75M'
#         elif pmv <= 74.99:
#             max_price = str(pmim+1)+'M'
#         else:
#             max_price = str(pmim+1)+'.25M'
#         url = ',max-price='+max_price

#     elif price <= 3250000:
#         if pmvm <= 24.99:
#             max_price = str(pmim)+'.75M'
#         elif pmvm <= 49.99:
#             max_price = str(pmim+1)+'M'
#         elif pmvm <= 74.99:
#             max_price = str(pmim+1)+'.25M'
#         else:
#             max_price = str(pmim+1)+'.5M'
#         url = ',max-price='+max_price

#     elif price <= 3500000:
#         url = ',min-price=2.75M,max-price=4.25M'
#     elif price <= 4000000:
#         url = ',min-price=3.25M,max-price=5M'
#     elif price <= 4500000:
#         url = ',min-price=3.5M,max-price=6M'
#     elif price <= 5000000:
#         url = ',min-price=4.25M,max-price=7M'
#     elif price <= 6000000:
#         url = ',min-price=4.5M,max-price=8M'
# #     elif price <= 7000000:
# #         url = ',min-price=5M'
# #     elif price <= 9000000:
# #         url = ',min-price=5M'
# #     else:
# #         url = ',min-price=6M'
#     return url

In [37]:
def search_url_part_four_gen(price):

    if sqft <= 350:
        url = ',max-sqft=500'

    elif sqft <= 550:
        url = ',max-sqft=500'

    elif sqft <= 650:
        url = ',min-sqft=500,max-sqft=750'

    elif sqft <= 850:
        url = ',min-sqft=500,max-sqft=1000'

    elif sqft <= 950:
        url = ',min-sqft=750,max-sqft=1000'

    elif sqft <= 1100:
        url = ',min-sqft=750,max-sqft=1250'

    elif sqft <= 1300:
        url = ',min-sqft=1000,max-sqft=1500'

    elif sqft <= 1600:
        url = ',min-sqft=1250,max-sqft=1750'

    elif sqft <= 1800:
        url = ',min-sqft=1500,max-sqft=2000'

    elif sqft <= 2000:
        url = ',min-sqft=1500,max-sqft=2250'

    elif sqft <= 2200:
        url = ',min-sqft=1750,max-sqft=2500'

    elif sqft <= 2500:
        url = ',min-sqft=2000,max-sqft=2750'

    elif sqft <= 2750:
        url = ',min-sqft=2250,max-sqft=3000'

    elif sqft <= 3000:
        url = ',min-sqft=2500,max-sqft=3500'

    elif sqft <= 3500:
        url = ',min-sqft=3000,max-sqft=4000'

    elif sqft <= 4000:
        url = ',min-sqft=3000,max-sqft=4500'

    elif sqft <= 4500:
        url = ',min-sqft=3500,max-sqft=5000'

    elif sqft <= 5000:
        url = ',min-sqft=4000,max-sqft=7500'

    elif sqft <= 6000:
        url = ',min-sqft=4500,max-sqft=7500'

    elif sqft <= 6500:
        url = ',min-sqft=5000,max-sqft=7500'

    elif sqft <= 7500:
        url = ',min-sqft=5000'

    else:
        url = ',min-sqft=7500'

    return url

In [19]:
zip_code = int(customer_df['zip'].values)
city = str(customer_df['city'].values)
price = float(customer_df['price'].values)
sqft = int(customer_df['sqft'].values)
num_bds = int(customer_df['num_bds'].values)
num_bths = float(customer_df['num_bths'].values)
lot_sqft = int(customer_df['lot_sf'].values)
yr_blt = int(customer_df['yr_blt'].values)
hoa = customer_df['HOA'].values
hoa_fee = float(customer_df['HOA_fee'].values)
type_home = (customer_df['home_type'].values)


url_part_one = 'https://www.redfin.com/zipcode/'+str(zip_code)+'/filter/sort=lo-days/'

url_part_two = search_url_part_two_gen(type_home)

url_part_three = search_url_part_three_gen(price)

url_part_four = 

url_part_five = 

url_part_six = 

url_part_seven = 

url_part_eight = 

url_part_nine = 





if sqft <= 300:
    max_sqft = '500'
elif sqft <= 500:
    max_sqft = '750'
elif sqft <= 750:
    max_sqft = '1k'
elif sqft <= 2800:
    i = sqft//1000
    ii = sqft % 1000
    if ii <= 250:
        max_sqft = str(i)+'.5k'
    elif ii <= 500:
        max_sqft = str(i)+'.75k'
    elif ii <= 750:
        max_sqft = str(i+1)
    else:
        max_sqft = str(i+1)+'.25k'
elif sqft <= 3100:
    max_sqft = '3.5k'
elif sqft <= 3400:
    max_sqft = '4k'
elif sqft <= 3750:
    max_sqft = '4.5k'
elif sqft <= 4100:
    max_sqft = '5k'
elif sqft <= 5500:
    max_sqft = '7.5k'
else:
    max_sqft = ''

url_part_four = ',max-sqft=500-sqft'
elif sqft <= 400:
    url_part_four = ',max-sqft=750-sqft'
elif sqft <= 500:
    url_part_four = ',min-sqft=500-sqft,max-sqft=1k-sqft'
elif sqft <= 600:
    url_part_four = ',min-sqft=750-sqft,max-sqft=1.5k-sqft'
elif sqft <= 700:
    url_part_four = ',min-sqft=1k-sqft,max-sqft=1.5k-sqft'
elif sqft <= 800:
    url_part_four = ',min-sqft=1.25k-sqft,max-sqft=2k-sqft'
elif sqft <= 900:
    url_part_four = ',min-sqft=1.5-sqft,max-sqft=2k-sqft'
elif sqft <= 1000:
    url_part_four = ',min-sqft=1.75-sqft,max-sqft=2.25k-sqft'
elif sqft <= 1100:
    url_part_four = ',min-sqft=500-sqft,max-sqft=2.5k-sqft'
elif sqft <= 1200:
    url_part_four = ',min-sqft=500-sqft,max-sqft=2.75k-sqft'
elif sqft <= 1300:
    url_part_four = ',min-sqft=500-sqft,max-sqft=3k-sqft'
elif sqft <= 1400:
    url_part_four = ',min-sqft=500-sqft,max-sqft=3.5k-sqft'
elif sqft <= 1500:
    url_part_four = ',min-sqft=500-sqft,max-sqft=4k-sqft'
elif sqft <= 1600:
    url_part_four = ',min-sqft=500-sqft,max-sqft=5k-sqft'
elif sqft <= 1700:
    url_part_four = ',min-sqft=500-sqft,max-sqft=7.5k-sqft'


search_url =

1.25k
-sqft
, min-year-built =
1960
, min-lot-size =
6.5k-sqft
, hoa =
400

In [ ]:
home_facts = =""

In [15]:
active_zip_url_list = gen_active_url_list(zip_)
df = pd.DataFrame(columns=['full_address', 'home_link'])
active_zip_url_list, df, proxies = links_for_active_props(proxies, active_zip_url_list, df, ua)
# for url in df['home_link']:
#     active_df = each_property(url, header, proxy)
# return active_df[0:2]